In [1]:
# Importing  all the functions defined in functions.py
from functions_try import *

In [25]:
# from numpy import * 

# model setup
n_iter = 1
t_max = 150 # how many days will the simulation run
n_hh = 1000
type_of_hh_array = [0, 1, 2]
prob_type_of_hh_array = [0.33, 0.33, 0.34]
mean_hh_size_array = [5, 5, 5]
initial_prob_I_array = [0.01, 0.01, 0.01]
mean_n_contacts_within_area = [2, 2, 4]
mean_n_contacts_outside_area = [3, 3, 3]
mean_n_contacts_outside_hh = np.array([[3, 2, 2], [2, 3, 2], [2, 2, 3]])

# transmission parameters
max_recovery_t = 10 # assuming person definitely recovers by these many days
p_max = 0.02 # probability of infecting someone upon contact on most infectious day
mean_gamma = 3.86
sd_gamma = 2.65


In [26]:
# calculating probability of infecting someone upon contact given the no. of days since infection.
p_t = cal_prob_of_inf(p_max, mean_gamma, sd_gamma, t_inf_max = max_recovery_t)

#define x-axis values
t = np.linspace (0, max_recovery_t, max_recovery_t+1) 

# import matplotlib.pyplot as plt
# #create plot of Gamma distribution
# plt.plot(t, p_t)

In [27]:
list_hh_ind_input = create_hh(n_hh_input = n_hh, type_of_hh_array_input = type_of_hh_array, \
                              prob_type_of_hh_array_input = prob_type_of_hh_array, \
                              mean_hh_size_array_input = mean_hh_size_array, \
                              initial_prob_I_array_input = initial_prob_I_array)

n_ind_input = len(list_hh_ind_input[1])

In [28]:
print("no. of inds in area 0 ", len(list_hh_ind_input[2][0]))
print("no. of inds in area 1 ", len(list_hh_ind_input[2][1]))
print("no. of inds in area 2 ", len(list_hh_ind_input[2][2]))

no. of inds in area 0  1735
no. of inds in area 1  1525
no. of inds in area 2  1740


In [29]:
import networkx as nx
import matplotlib as plt

# creating connections between members of the same household
# initialising graph
graph = nx.Graph()

for hh in range(n_hh):
    inds_temp = list_hh_ind_input[0][hh].get_individuals()
    graph.add_nodes_from(inds_temp)
    graph.add_edges_from(itertools.combinations(inds_temp, 2))
    

# list(graph.edges)
# graph.degree[ind]

# ADD THIS TO CREATING INDIVIDUALS FUNCTION
# Adding attributes about how many contacts will people have within and outside their area
for ind in list_hh_ind_input[1]:
    area_temp = ind.get_type_of_hh()
    # ADD POISSON DRAW HERE
    temp_within_area_contacts = mean_n_contacts_within_area[area_temp] 
    ind.add_n_within_area_contacts(temp_within_area_contacts)
    # ADD POISSON DRAW HERE
    temp_outside_area_contacts = mean_n_contacts_outside_area[area_temp] 
    ind.add_n_outside_area_contacts(temp_outside_area_contacts)    

# creating connections between members of the same area 
# (this can take very long or run into problems with small sample sizes)   
for area in range(len(type_of_hh_array)):
    for ind1 in list_hh_ind_input[2][area]:   
        # choosing ind1 from the list of individuals in a given area
        ind1_degree_hh = ind1.get_hh_size() - 1 # this is fixed number for an individual
        ind1_degree_hh_within_area = ind1_degree_hh + ind1.get_n_within_area_contacts() # this is fixed number for an individual

        # don't run the following if the maximum possible connections for ind1 has been reached
        while graph.degree[ind1] < ind1_degree_hh_within_area:    
            # drawing a potential contact from within the area
            ind2  = random.sample(list_hh_ind_input[2][area], 1)[0]
            # checking how many contacts this person can have (in hh and within area)
            ind2_degree_hh = ind2.get_hh_size() - 1
            ind2_degree_hh_within_area = ind2_degree_hh + ind2.get_n_within_area_contacts()
            # check if this person is NOT from same hh, or already connected, or has not reached maximum possible connections setup
            if (ind2.get_hh() != ind1.get_hh()) & (ind2 not in graph[ind1]) & (graph.degree[ind2] < ind2_degree_hh_within_area):
                graph.add_edge(ind1, ind2)


            
for area in range(len(type_of_hh_array)-1):  
    # Outside area contacts will be done sequentially according to area
    # making a list of individuals from the same area and those that have all their contact set - hence Do Not Disturb
    list_ind_DND = []
    if area > 0:
        areas_DND = range(0, area + 1) # areas before this area and current (within) cannot be disturbed (connections)
        for area_DND in areas_DND:
            list_ind_DND =  list_ind_DND + list_hh_ind_input[2][area_DND]

    # looping through individuals    
    for ind1 in list_hh_ind_input[2][area]:
        ind1_degree_hh = ind1.get_hh_size() - 1 # this is fixed number for an individual
        ind1_degree_hh_within_area = ind1_degree_hh + ind1.get_n_within_area_contacts() # this is fixed number for an individual
        ind1_degree_hh_within_outside_area = ind1_degree_hh_within_area + ind1.get_n_outside_area_contacts() # this is fixed number for an individual
    #     print("ind" , ind1.get_ID(), "can have hh contacts = ", ind1_degree_hh)
    #     print("ind" , ind1.get_ID(), "can have hh+within contacts = ", ind1_degree_hh_within_area)
    #     print("ind" , ind1.get_ID(), "can have MAX contacts = ", ind1_degree_hh_within_outside_area)

        # don't run the following if the maximum possible connections for ind1 has been reached
        while graph.degree[ind1] < ind1_degree_hh_within_outside_area:
            # drawing a potential contact from outside the area
            list_inds_otherarea = list(set(list_hh_ind_input[1]) - set(list_ind_DND))
            print(len(list_inds_otherarea))
            ind2  = random.sample(list_inds_otherarea, 1)[0]
            # checking how many contacts this person can have (in hh and within area)
            ind2_degree_hh = ind2.get_hh_size() - 1
            ind2_degree_hh_within_area = ind2_degree_hh + ind2.get_n_within_area_contacts()
            ind2_degree_hh_within_outside_area = ind2_degree_hh_within_area + ind2.get_n_outside_area_contacts() # this is fixed number for an individual
            # check if this person is NOT already connected, or has not reached maximum possible connections setup
            if (ind2 not in graph[ind1]) & (graph.degree[ind2] < ind2_degree_hh_within_outside_area):
                graph.add_edge(ind1, ind2)
    
    

5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740


1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740


1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740
1740


In [30]:
for i in list_hh_ind_input[2][2]:
    print(graph.degree[i]) 

9
11
11
10
10
9
11
11
11
10
11
11
11
11
11
11
11
11
10
11
11
11
11
11
9
11
10
11
11
10
9
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
10
11
11
11
11
11
11
11
11
11
11
11
11
10
11
11
11
11
11
11
11
9
11
10
11
9
11
11
11
11
11
11
11
11
11
9
11
11
11
11
11
11
11
11
11
11
9
11
11
11
11
11
11
11
11
11
11
11
11
11
8
11
11
11
11
11
11
11
11
9
10
11
11
9
10
11
11
11
11
9
9
11
11
11
11
11
10
11
11
11
10
11
11
11
10
11
11
11
11
11
11
10
11
11
11
11
11
11
11
11
10
11
11
11
10
11
11
11
11
9
11
11
11
11
10
11
11
10
11
11
8
9
11
11
11
11
10
11
11
11
11
11
11
11
11
10
11
11
11
10
11
11
10
11
11
11
10
11
11
11
9
10
11
11
9
11
10
11
11
11
10
11
10
11
9
11
10
11
11
11
11
10
11
11
11
11
11
11
11
11
11
11
11
11
10
10
11
11
11
11
11
11
11
11
11
9
11
11
11
11
10
11
11
11
11
11
11
11
11
11
11
11
11
11
10
11
11
9
11
11
11
11
9
11
11
11
11
11
11
8
11
11
8
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
10
11
11
11
11
9
11
11
10
11
11
11
11
11
11
11
11
11
10
11
11
11
11
11
11
11
10
11
11
11
11
10
9
11
11
11
11


In [ ]:
# calculating total number of individuals created
n_ind_input = len(list_hh_ind_input[1])

# creating the initial adjacency list for this population
graph = create_adjacency_list(list_hh_ind_input = list_hh_ind_input, n_hh_input = n_hh, 
                                    n_ind_input = n_ind_input, \
                                    type_of_hh_array_input = type_of_hh_array, \
                                    mean_n_contacts_within_area_input = mean_n_contacts_within_area, \
                                    mean_n_contacts_outside_area_input = mean_n_contacts_outside_area)


In [ ]:
len(list_hh_ind_input[2][0])

In [ ]:
for i in range(len(list_hh_ind_input[2])):
    print(i)

In [ ]:
for area_list in list_hh_ind_input[2]:
    for i in area_list:
        print("HELLO")
        print(i)


In [ ]:
# running the simulations and the output is saved in csv files
sim(n_iter_input = n_iter, n_hh_input = n_hh, type_of_hh_array_input = type_of_hh_array, \
    prob_type_of_hh_array_input = prob_type_of_hh_array, mean_hh_size_array_input = mean_hh_size_array, \
    initial_prob_I_array_input = initial_prob_I_array, \
    mean_n_contacts_within_area_input = mean_n_contacts_within_area, \
    mean_n_contacts_outside_area_input = mean_n_contacts_outside_area, \
    t_max_input = t_max, max_recovery_t_input = max_recovery_t, p_t_input = p_t)

In [ ]:
# importing outputs
import pandas as pd

dat = pd.read_csv('sim.csv')

In [ ]:
# ploting S, I, R dynamics over time
import matplotlib.pyplot as plt

for i in range(1,n_iter+1): 
    dat_temp = dat.loc[dat['iter '] == i]
    
    # x axis values
    x = dat_temp[['time']]
    # corresponding y axis values
    S = dat_temp[['S']]
    I = dat_temp[['I']]
    R = dat_temp[['R']]

    # plotting the points
    plt.plot(x, S, 'y', label = "Susceptibles")
    plt.plot(x, I, 'r', label = "Infected")
    plt.plot(x, R, 'g', label = "Recovered")


    # naming the x axis
    plt.xlabel('Time')
    # naming the y axis
    plt.ylabel('y - axis')

# giving a title to my graph
plt.title('Number of individuals in states over time')
# show a legend on the plot
plt.legend(['Susceptible','Infected','Recovered'])
# function to show the plot
plt.show()



In [ ]:
import pandas as pd

colnames = ['infector_ID', 'infector_type_of_hh', 'infectee_ID', 'infectee_type_of_hh', 'time', 'iter']
df = pd.read_csv('test.csv', names = colnames)
df = df.iloc[1: , :] # dropping first row (these are column headings now)
df = df.loc[df['iter'] == '1']

In [ ]:
import networkx as nx
import matplotlib as plt

# fig, ax = plt.subplots(figsize=(15, 8))
# initialising graph
I = nx.Graph()

# creating edge list (reading it from the output)
edge_list = list(df[['infector_ID', 'infectee_ID']].apply(tuple, axis=1))
I.add_edges_from(edge_list)

df1 = df[['infector_ID', 'infector_type_of_hh']]
df1.columns = ['node', 'type_of_hh']
df2 = df[['infectee_ID', 'infectee_type_of_hh']]
df2.columns = ['node', 'type_of_hh']

frames = [df1, df2]

df3 = pd.concat(frames)
df3 = df3.drop_duplicates()

car = df3

# Make types into categories
car = car.set_index('node')

car['type_of_hh'] = pd.Categorical(car['type_of_hh'])

 
# Specify colors
cmap = plt.colors.ListedColormap(['C0', 'darkorange', 'green'])

# drawing network
nx.draw(I, with_labels=False, node_color=car['type_of_hh'].cat.codes, cmap = cmap)



In [ ]:
# printing output of contacts 
list_ind = []
list_type_of_hh = []
list_hh_size = []
list_n_same_hh = []
list_n_same_area = []
list_n_outside_area = []
list_n_total = []

for i in range(n_ind_input):
    ind_temp = list(graph.m_adj_list.keys())[i].get_ID()
    list_ind.append(ind_temp)
    
    type_of_hh_temp = list(graph.m_adj_list.keys())[i].get_type_of_hh()
    list_type_of_hh.append(type_of_hh_temp) 
    
    hh_size_temp = int(list(graph.m_adj_list.keys())[i].get_hh_size())
    list_hh_size.append(hh_size_temp)
       
    n_same_hh_temp = len(list(filter(lambda x: x[1] == "same_hh", graph.m_adj_list[list(graph.m_adj_list.keys())[i]])))
    list_n_same_hh.append(n_same_hh_temp)
    
    n_same_area_temp = len(list(filter(lambda x: x[1] == "same_area", graph.m_adj_list[list(graph.m_adj_list.keys())[i]])))
    list_n_same_area.append(n_same_area_temp)
    
    n_outside_area_temp = len(list(filter(lambda x: x[1] == "outside_area", graph.m_adj_list[list(graph.m_adj_list.keys())[i]])))
    list_n_outside_area.append(n_outside_area_temp)
    
    n_total_temp = len(graph.m_adj_list[list(graph.m_adj_list.keys())[i]])
    list_n_total.append(n_total_temp)

    
contact_list = [list_ind, list_type_of_hh, list_hh_size, list_n_same_hh, list_n_same_area, list_n_outside_area, list_n_total]


# Convert output to a dataframe
import pandas as pd

contact_list_df = []
contact_list_df = pd.DataFrame(contact_list).transpose()
contact_list_df.rename(columns={0:"ID", 1: "type_of_hh", 2:"hh_size", 3:"contacts_within_hh", 4:"contacts_within_area", 5:"contacts_outside_area", 6: "contacts_total"}, inplace = True)

pd.set_option("display.max_rows", None, "display.max_columns", None)

display(contact_list_df)



In [ ]:
from matplotlib import pyplot as plt

groups = contact_list_df.groupby("type_of_hh")
for name, group in groups:
    plt.hist(group["hh_size"], label=name, alpha = 0.5)
    plt.xlabel('Household size')
    plt.ylabel('Frequency')
plt.legend()